In [1]:
import os
from glob import glob
from pprint import pprint

import pandas as pd
from config import SETTINGS
from pyodm import Node

import seabeepy as sb

# Check mission progress

Checks processing status of missions submitted to the SeaBee platform.

## 1. Tasks on NodeODM

In [2]:
node = Node.from_url("http://nodeodm")
print(node.info())

nodeodm_task_ids = node.get("task/list")
task_list = [node.get_task(t["uuid"]) for t in nodeodm_task_ids]
print("Total number of tasks:", len(task_list))

task_df = pd.DataFrame(
    [
        (
            idx,
            t.info().uuid,
            os.path.split(t.info().name)[1],
            t.info().images_count,
            t.info().progress,
            str(t.info().status).split(".")[1],
            t.output()[-1]
        )
        for idx, t in enumerate(task_list)
    ],
    columns=["task_index", "uuid", "mission_name", "n_images", "progress", "status", "last_message"],
)
task_df

{'version': '2.2.2', 'task_queue_count': 0, 'total_memory': 540553396224, 'available_memory': 385196969984, 'cpu_cores': 256, 'max_images': None, 'max_parallel_tasks': 4, 'engine': 'odm', 'engine_version': '3.1.7', 'odm_version': '?'}
Total number of tasks: 1


,task_index,uuid,mission_name,n_images,progress,status,last_message
0,0,854ab8ae-a557-4b1b-8f80-6c048ef0f311,agder_songvaar_20230524,1773,100,FAILED,[ERROR] Uh oh! Processing stopped because of...


### Task output

Check output of running tasks, could perhaps just look at one task since the output is quite big.

In [7]:
for task in task_list:
    if str(task.info().status) == "TaskStatus.FAILED":
        print(f"Task {task.info().name} at {task.info().progress}%")
        for l in task.output()[-30:]:
            print(l)
        print("\n")

Task /home/notebook/shared-seabee-ns9879k/seabirds/2023/agder_songvaar_20230524 at 100%
File "/code/SuperBuild/install/bin/opensfm/opensfm/io.py", line 1469, in open
return open(*args, **kwargs)
OSError: [Errno 116] Stale file handle: '/var/www/data/854ab8ae-a557-4b1b-8f80-6c048ef0f311/opensfm/exif/DJI_20230524115751_0178_V.JPG.exif'

===== Dumping Info for Geeks (developers need this to fix bugs) =====
Child returned 1
Traceback (most recent call last):
File "/code/stages/odm_app.py", line 81, in execute
self.first_stage.run()
File "/code/opendm/types.py", line 398, in run
self.next_stage.run(outputs)
File "/code/opendm/types.py", line 398, in run
self.next_stage.run(outputs)
File "/code/opendm/types.py", line 398, in run
self.next_stage.run(outputs)
File "/code/opendm/types.py", line 377, in run
self.process(self.args, outputs)
File "/code/stages/run_opensfm.py", line 38, in process
octx.reconstruct(args.rolling_shutter, reconstruction.is_georeferenced() and (not args.sfm_no_partial)

### Restart failed tasks

Restart task that was previously canceled, that had failed to process or that successfully completed. We just restart failed tasks.

In [ ]:
failed_tasks = [t for t in task_list if t.info().status == "TaskStatus.FAILED"]
set([t.info().last_error for t in failed_tasks])

In [ ]:
for task in failed_tasks:
    if task.info().last_error == "Cannot process dataset":
        task.restart()

In [3]:
# print(task_list[0].info())
# sorted([n.name for n in node.options()])
# task_list[0].cancel()
# task_list[0].remove()

## 2. Folder structure

In [4]:
# User input
base_dir = r"/home/notebook/shared-seabee-ns9879k/seabirds/2023/"

In [5]:
%%capture

# Check mission data in 'base_dir'
dir_list = sorted(glob(os.path.join(base_dir, "*/")))
dir_list = [os.path.normpath(f) for f in dir_list]

names = [f for f in dir_list if sb.ortho.parse_mission_data(os.path.basename(f))]
bad_names = [
    f for f in dir_list if not sb.ortho.parse_mission_data(os.path.basename(f))
]

configs = [f for f in dir_list if sb.ortho.check_config_exists(f)]
no_configs = [f for f in dir_list if not sb.ortho.check_config_exists(f)]

valid_configs = [f for f in configs if sb.ortho.check_config_valid(f)]
invalid_configs = [f for f in configs if not sb.ortho.check_config_valid(f)]

images = [f for f in dir_list if sb.ortho.check_subdir_exists(f, "images")]
no_images = [f for f in dir_list if not sb.ortho.check_subdir_exists(f, "images")]

orthos = [f for f in dir_list if sb.ortho.check_subdir_exists(f, "orthophoto")]
no_orthos = [f for f in dir_list if not sb.ortho.check_subdir_exists(f, "orthophoto")]

images_configs = [f for f in valid_configs if f in images]
valid_counts = [f for f in images_configs if sb.ortho.check_file_count(f)]
invalid_counts = [f for f in images_configs if not sb.ortho.check_file_count(f)]

to_process = [
    f
    for f in valid_counts
    if not sb.ortho.check_subdir_exists(f, "orthophoto") and f in names
]

no_mosaic = [f for f in to_process if not sb.ortho.parse_config(f)["mosaic"]]
no_publish = [f for f in to_process if not sb.ortho.parse_config(f)["publish"]]

In [6]:
print("Scanning", base_dir)
print(len(dir_list), "sub-directories in total.")
print(len(names), f"have valid mission names ({len(bad_names)} are invalid).")
print(
    len(valid_configs),
    f"directories have valid config. files ({len(no_configs)} have no config. and {len(invalid_configs)} are invalid).",
)
print(
    len(images), f"directories have 'images' sub-directories ({len(no_images)} do not)."
)
print(
    len(orthos),
    f"directories have 'orthophoto' sub-directories ({len(no_orthos)} do not).",
)
print(
    len(images_configs),
    f"have both images and valid configs. {len(valid_counts)} of these have the correct number of images and {len(invalid_counts)} do not.",
)
print(
    len(to_process),
    f"missions can be processed ({len(no_mosaic)} have 'mosaic=False' and {len(no_publish)} have 'publish=False').",
)

print(f"\nThe following {len(bad_names)} folders have invalid names:")
pprint(bad_names)

print(f"\nThe following {len(no_configs)} folders have no config.:")
pprint(no_configs)

print(f"\nThe following {len(invalid_configs)} folders have invalid config.:")
pprint(invalid_configs)

print(f"\nThe following {len(no_images)} folders have no images:")
pprint(no_images)

print(f"\nThe following {len(invalid_counts)} folders have invalid image counts:")
pprint(invalid_counts)

print(f"\nThe following {len(to_process)} folders are ready to be processed:")
pprint(to_process)

print(f"\nOf these, the following {len(no_mosaic)} folders have 'mosaic=False':")
pprint(no_mosaic)

print(f"\nand the following {len(no_publish)} folders have 'publish=False':")
pprint(no_publish)

Scanning /home/notebook/shared-seabee-ns9879k/seabirds/2023/
444 sub-directories in total.
442 have valid mission names (2 are invalid).
442 directories have valid config. files (2 have no config. and 0 are invalid).
443 directories have 'images' sub-directories (1 do not).
433 directories have 'orthophoto' sub-directories (11 do not).
442 have both images and valid configs. 442 of these have the correct number of images and 0 do not.
9 missions can be processed (1 have 'mosaic=False' and 1 have 'publish=False').

The following 2 folders have invalid names:
['/home/notebook/shared-seabee-ns9879k/seabirds/2023/Karmoy_Jarstein_Sor_midt-20230514',
 '/home/notebook/shared-seabee-ns9879k/seabirds/2023/nonmission']

The following 2 folders have no config.:
['/home/notebook/shared-seabee-ns9879k/seabirds/2023/Karmoy_Jarstein_Sor_midt-20230514',
 '/home/notebook/shared-seabee-ns9879k/seabirds/2023/nonmission']

The following 0 folders have invalid config.:
[]

The following 1 folders have no i

In [7]:
# Missions with non-standard ODM settings
[f for f in valid_configs if sb.ortho.parse_config(f).get("odm_options")]

['/home/notebook/shared-seabee-ns9879k/seabirds/2023/Bergen_Realfagstaket_20230530']